In [1]:
from kafka import KafkaProducer,KafkaClient
from kafka.errors import KafkaError
import time
import json
import websocket
import logging
import re
#LOCAL IMPORTS
import config as myCred

In [11]:
#functions for API connnection

startTime =time.time()

def onOpen(ws):
    print("connection Opened")
    auth_data={
        "action": "authenticate",
        "data": {"key_id": myCred.API_KEY, "secret_key": myCred.SECRET_KEY}
    }
    
    ws.send(json.dumps(auth_data))
    
    listenMsg = {
        "action": "listen", 
        "data": {"streams": ["T.TSLA","T.AAPL","T.MSFT"]}
    }

    ws.send(json.dumps(listenMsg))
        
def onMsg(ws,message):
    if re.match('^.*authorized.*$',message):
        return
    else:
        if(time.time()-startTime)<15:
            print(message)
            try:
                producer.send(topicName, message.encode('utf-8'))
                return True
            except Exception as ex:
                print('Exception in publishing message')
                print(str(ex))
        else:
            ws.close()
    
def onClose(ws):
    print("connection Closed")

In [12]:
socket = "wss://data.alpaca.markets/stream"
ws = websocket.WebSocketApp(socket,on_open=onOpen, on_close=onClose, on_message=onMsg)
try:
    producer=KafkaProducer(bootstrap_servers=" localhost:9092")
    topicName="stockData"
except KafkaError:
    print(logging.exception("no broker available"))
    

In [13]:
ws.run_forever()

connection Opened
{"stream":"listening","data":{"streams":["T.TSLA","T.AAPL","T.MSFT"]}}
{"stream":"T.AAPL","data":{"ev":"T","T":"AAPL","i":"1830","x":2,"p":365.5,"s":8,"t":1593532885446000000,"c":[14,37,41],"z":3}}
{"stream":"T.AAPL","data":{"ev":"T","T":"AAPL","i":"1831","x":2,"p":365.5,"s":92,"t":1593532885446000000,"c":[37],"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"2320","x":17,"p":1080.54,"s":70,"t":1593532886047000000,"c":[37],"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"2321","x":17,"p":1080.54,"s":30,"t":1593532886047000000,"c":[14,37,41],"z":3}}
{"stream":"T.MSFT","data":{"ev":"T","T":"MSFT","i":"1536","x":15,"p":201.325,"s":100,"t":1593532886130000000,"c":null,"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"2322","x":17,"p":1080.55,"s":100,"t":1593532886362000000,"c":null,"z":3}}
{"stream":"T.TSLA","data":{"ev":"T","T":"TSLA","i":"1245","x":15,"p":1080.365,"s":20,"t":1593532887696000000,"c":[37],"z":3}}
{"stream":"T.TSLA","data":{"